Целью является создание модели прогнозирования энергопотребления просьюмеров для снижения затрат на небаланс энергии.

Проект направлен на решение проблемы энергетического дисбаланса - ситуации, когда ожидаемое потребление энергии не совпадает с ее фактическим использованием или производством. Значительная часть энергетического дисбаланса приходится на потребителей, которые как потребляют, так и производят энергию. Несмотря на то, что они составляют лишь небольшую часть всех потребителей, их непредсказуемое потребление энергии создает логистические и финансовые проблемы для энергетических компаний.

a person who buys electronic goods that are of a standard between those aimed at consumers and professionals.
"the magazine is aimed at the prosumer who uses a £5,000 camera to make home movies of his dog"
2.
a consumer who becomes involved with designing or customizing products for their own needs.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

# запишем путь директории
basedir = os.path.abspath(os.getcwd())

'C:\\Users\\Марат Жанпейсов\\Downloads\\Projects\\enefit'

In [8]:
# загрузим данные
try:
    clients = pd.read_csv(os.path.join(basedir, 'data/client.csv'))
    train = pd.read_csv(os.path.join(basedir, 'data/train.csv'))
    gas_prices = pd.read_csv(os.path.join(basedir, 'data/gas_prices.csv'))
    electricity_prices = pd.read_csv(os.path.join(basedir, 'data/electricity_prices.csv'))
    forecast_weather = pd.read_csv(os.path.join(basedir, 'data/forecast_weather.csv'))
except:
    raise Exception('Check the files')